In [3]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv('Egypt_terr.csv')
data.head()

,Unnamed: 0,eventid,iyear,imonth,iday,extended,country,country_txt,region,region_txt,...,weaptype1,weaptype1_txt,nkill,property,ishostkid,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
0,133,197003140002,1970,3,14,0,60,Egypt,10,Middle East & North Africa,...,6,Explosives,0.0,1,0.0,PGIS,-9,-9,0,-9
1,1092,197111280001,1971,11,28,0,60,Egypt,10,Middle East & North Africa,...,5,Firearms,1.0,0,0.0,PGIS,1,1,0,1
2,1181,197202190001,1972,2,19,0,60,Egypt,10,Middle East & North Africa,...,6,Explosives,0.0,0,1.0,Hijacking DB,0,1,1,1
3,5603,197712050001,1977,12,5,0,60,Egypt,10,Middle East & North Africa,...,5,Firearms,1.0,0,0.0,PGIS,-9,-9,1,1
4,6797,197809300002,1978,9,30,0,60,Egypt,10,Middle East & North Africa,...,6,Explosives,0.0,1,0.0,PGIS,1,0,1,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2479 non-null   int64  
 1   eventid           2479 non-null   int64  
 2   iyear             2479 non-null   int64  
 3   imonth            2479 non-null   int64  
 4   iday              2479 non-null   int64  
 5   extended          2479 non-null   int64  
 6   country           2479 non-null   int64  
 7   country_txt       2479 non-null   object 
 8   region            2479 non-null   int64  
 9   region_txt        2479 non-null   object 
 10  provstate         2461 non-null   object 
 11  city              2478 non-null   object 
 12  latitude          2458 non-null   float64
 13  longitude         2458 non-null   float64
 14  specificity       2479 non-null   float64
 15  vicinity          2479 non-null   int64  
 16  crit1             2479 non-null   int64  


In [6]:
def missingvaluesdf():

    missing_values=pd.DataFrame(data.isna().sum())

    filt=(missing_values.iloc[:,0]>0)
    return missing_values[filt]

Let's explore more in detail the missing data

In [7]:
data.loc[data.provstate.isna()==True][['provstate','city']]

,provstate,city
5,NaN,Unknown
7,NaN,Alexandria
21,NaN,Giza
58,NaN,Hurghada
66,NaN,Beni Suef
72,NaN,Damietta
109,NaN,Luxor
112,NaN,Faiyum
125,NaN,Port Said
163,NaN,Mir


In [8]:
#Having the provstate for each city would be a great help. Let's see if we can use the rest of the dataset for it
provcity=data[['provstate','city']].dropna(axis=0,how='any')


In [9]:
provcity.drop_duplicates(keep='first',ignore_index=True,inplace=True)

In [10]:
provcity.nunique()

provstate     43
city         327
dtype: int64

In [11]:
dict_city=provcity.set_index('city').to_dict()['provstate']

In [12]:
len(dict_city)

327

In [13]:
#Now we have a dictionary {city: provstate}, we should be able to fill in the missing values in provstate
data['provstate']=data['provstate'].fillna(data['city'].apply(lambda x: dict_city.get(x)))

In [14]:
missing_values=pd.DataFrame(data.isna().sum())

filt=(missing_values.iloc[:,0]>0)
missing_values[filt]

,0
provstate,4
city,1
latitude,21
longitude,21
targsubtype1,238
targsubtype1_txt,238
target1,5
nkill,33


In [15]:
#4 values missing now in provstate. Let's see if we can fill them in manually, or if we just drop them
data.loc[data['provstate'].isna()==True][['city','latitude','longitude']]

,city,latitude,longitude
163,Mir,27.441690,30.746817
219,Abu Mawas,27.641389,30.849444
245,Izbat Sayk Basha,30.758611,31.735833
248,Idfa,26.571904,31.638356


In [16]:
#4 keys are missing, we will add them manually and update our dictionary
dict_city2={'Mir':'Asyut','Abu Mawas':'Minya','Izbat Sayk Basha':'Al Sharqia','Idfa':'Sohag'}
dict_city.update(dict_city2)

In [17]:
data['provstate']=data['provstate'].fillna(data['city'].apply(lambda x: dict_city.get(x)))

In [18]:
missing_values=pd.DataFrame(data.isna().sum())

filt=(missing_values.iloc[:,0]>0)
missing_values[filt]

,0
city,1
latitude,21
longitude,21
targsubtype1,238
targsubtype1_txt,238
target1,5
nkill,33


In [19]:
data.loc[data['city'].isna()==True][['city','latitude','longitude']]

,city,latitude,longitude
496,NaN,30.608472,33.617577


In [20]:
#The city is Al Hasna, let's fill it manually
data['city']=data['city'].fillna(value='Al Hasna')

In [21]:

missing_values=pd.DataFrame(data.isna().sum())

filt=(missing_values.iloc[:,0]>0)
missing_values[filt]

,0
latitude,21
longitude,21
targsubtype1,238
targsubtype1_txt,238
target1,5
nkill,33


In [22]:
#Let's check latitude and longitude
data.loc[data.latitude.isna()==True]['city']

68             Unknown
76             Unknown
79             Unknown
138        Upper Egypt
214     Southern Egypt
215     Southern Egypt
291         Edka-Hener
292              Hener
299              Esfay
582            Unknown
724            Unknown
725            Unknown
770            Unknown
779            Unknown
1057           Unknown
1086           Unknown
1172           Unknown
1173           Unknown
1610           Unknown
1620           Unknown
1836           Unknown
Name: city, dtype: object

In [23]:
#Even the cities are unknown. However, we don't know for sure it will prevent us from analyzing the dataset. We should maybe first 
#replace with "0" before analyzing locations

data[['latitude','longitude']]=data[['latitude','longitude']].fillna(value=0)

In [24]:
missing_values=pd.DataFrame(data.isna().sum())

filt=(missing_values.iloc[:,0]>0)
missing_values[filt]

,0
targsubtype1,238
targsubtype1_txt,238
target1,5
nkill,33


In [25]:
#Let's check targsubtype1 and targsubtype1_txt
data.loc[data.targsubtype1.isna()==True][['targtype1','targtype1_txt','targsubtype1','targsubtype1_txt']]

,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt
18,20,Unknown,NaN,NaN
45,2,Government (General),NaN,NaN
111,8,Educational Institution,NaN,NaN
273,17,Terrorists/Non-State Militia,NaN,NaN
276,17,Terrorists/Non-State Militia,NaN,NaN
...,...,...,...,...
2422,20,Unknown,NaN,NaN
2435,20,Unknown,NaN,NaN
2458,20,Unknown,NaN,NaN
2467,20,Unknown,NaN,NaN


In [26]:
data.loc[data.targtype1_txt=='Unknown'][['targtype1','targtype1_txt','targsubtype1','targsubtype1_txt']]['targtype1'].unique()

array([20], dtype=int64)

In [27]:
#The targtype 20 doesn't seem very useful, let's drop it
data=data.loc[data.targtype1!=20]

In [28]:
missingvaluesdf()

,0
targsubtype1,27
targsubtype1_txt,27
target1,5
nkill,32


In [29]:
data.shape

(2268, 45)

In [30]:
data.loc[data.targsubtype1_txt.isna()==True][['targtype1','targtype1_txt','targsubtype1','targsubtype1_txt']]

,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt
45,2,Government (General),NaN,NaN
111,8,Educational Institution,NaN,NaN
273,17,Terrorists/Non-State Militia,NaN,NaN
276,17,Terrorists/Non-State Militia,NaN,NaN
286,17,Terrorists/Non-State Militia,NaN,NaN
308,17,Terrorists/Non-State Militia,NaN,NaN
318,2,Government (General),NaN,NaN
369,1,Business,NaN,NaN
370,1,Business,NaN,NaN
371,1,Business,NaN,NaN


In [31]:
data.loc[data['targtype1_txt']=='Business']['targsubtype1_txt'].value_counts()

Retail/Grocery/Bakery                    48
Bank/Commerce                            20
Medical/Pharmaceutical                   12
Construction                             10
Farm/Ranch                                9
Hotel/Resort                              9
Restaurant/Bar/Café                       7
Gas/Oil/Electric                          7
Entertainment/Cultural/Stadium/Casino     5
Industrial/Textiles/Factory               2
Multinational Corporation                 2
Name: targsubtype1_txt, dtype: int64

In [32]:
data.loc[data['targtype1_txt']=='Telecommunication']['targsubtype1_txt']

809     Internet Infrastructure
853                         NaN
1041        Telephone/Telegraph
1319        Telephone/Telegraph
1392                        NaN
1632                        NaN
1694                        NaN
1695                        NaN
1706                        NaN
Name: targsubtype1_txt, dtype: object

In [33]:
data.loc[data['targtype1_txt']=='Terrorists/Non-State Militia']['targsubtype1_txt'].value_counts()

Terrorist            3
Non-State Militia    3
Name: targsubtype1_txt, dtype: int64

In [34]:
#We can see that for the columns targsubtype, the value can't really be guessed. We will fill the np.nan by 0 and Non available
data["targsubtype1"]=data["targsubtype1"].fillna(value=0)
data["targsubtype1_txt"]=data["targsubtype1_txt"].fillna(value='Not available')

In [35]:
missingvaluesdf()

,0
target1,5
nkill,32


In [36]:
#We're almost there! target1 and nkill now

data.loc[data.target1.isna()==True][['targtype1','targtype1_txt','targsubtype1','targsubtype1_txt']]

,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt
7,15,Religious Figures/Institutions,86.0,Place of Worship
8,15,Religious Figures/Institutions,86.0,Place of Worship
12,6,Airports & Aircraft,42.0,Aircraft (not at an airport)
23,11,Maritime,58.0,Commercial Maritime
239,2,Government (General),21.0,Government Building/Facility/Office


In [37]:
data.loc[data['targtype1']==15]['target1'].value_counts()

Church                                                  17
Mosque                                                  10
Coptic Christian Church                                  2
Shrine                                                   2
Former Grand Mufti: Sheikh Ali Gomaa                     1
Priest: Father Samaan Shehata                            1
Sufi Cleric: Sheikh Sulayman Abu-Haraz                   1
Saint Georges, the Two Saints and Abu Keer  churches     1
Two Saints Church                                        1
Coptic Bishop: Anba Makarios                             1
Religious Compound                                       1
group of members                                         1
Pharonic Temple                                          1
Church of Virgin Mary                                    1
Coptic Church                                            1
Sufi Leader: Sheikh Khaled al-Ratil Sawarka              1
Latin Church                                            

In [38]:
#Here as well, we can't guess... not available seems to be a right choice

data.target1=data.target1.fillna(value='Not available')

In [39]:
missingvaluesdf()

,0
nkill,32


In [40]:
data.loc[data.nkill.isna()==True]

,Unnamed: 0,eventid,iyear,imonth,iday,extended,country,country_txt,region,region_txt,...,weaptype1,weaptype1_txt,nkill,property,ishostkid,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
546,110960,201209160010,2012,9,16,0,60,Egypt,10,Middle East & North Africa,...,6,Explosives,NaN,-9,0.0,START Primary Collection,-9,-9,0,-9
573,116821,201305060031,2013,5,6,1,60,Egypt,10,Middle East & North Africa,...,5,Firearms,NaN,0,1.0,START Primary Collection,-9,-9,0,-9
582,118457,201306260039,2013,6,26,1,60,Egypt,10,Middle East & North Africa,...,5,Firearms,NaN,-9,1.0,START Primary Collection,-9,-9,0,-9
596,118740,201307050013,2013,7,5,0,60,Egypt,10,Middle East & North Africa,...,5,Firearms,NaN,0,0.0,START Primary Collection,-9,-9,0,-9
601,118786,201307060026,2013,7,6,0,60,Egypt,10,Middle East & North Africa,...,5,Firearms,NaN,-9,0.0,START Primary Collection,-9,-9,0,-9
654,119360,201307220043,2013,7,22,0,60,Egypt,10,Middle East & North Africa,...,5,Firearms,NaN,0,0.0,START Primary Collection,-9,-9,0,-9
709,120204,201308130032,2013,8,13,0,60,Egypt,10,Middle East & North Africa,...,6,Explosives,NaN,1,0.0,START Primary Collection,-9,-9,0,-9
722,120307,201308160037,2013,8,16,0,60,Egypt,10,Middle East & North Africa,...,6,Explosives,NaN,-9,0.0,START Primary Collection,-9,-9,0,-9
733,120482,201308220040,2013,8,22,0,60,Egypt,10,Middle East & North Africa,...,5,Firearms,NaN,1,0.0,START Primary Collection,-9,-9,0,-9
1046,134121,201406300028,2014,6,30,1,60,Egypt,10,Middle East & North Africa,...,5,Firearms,NaN,0,1.0,START Primary Collection,-9,-9,0,-9


In [41]:
#It is hard to find the number of casualties even in the press archives... given that it is only 32 lines over 2268, we can drop them
data=data.loc[data.nkill.isna()==False]

In [42]:
data.shape

(2236, 45)

In [43]:
missingvaluesdf()

,0


Now that we have carefully dealt with the missing values, let's continue with the cleaning

In [44]:
data.iday.unique()

array([14, 28, 19,  5, 30,  1,  6, 15, 10,  2,  7, 13, 24, 20, 23, 18, 27,
        3, 26,  4, 29,  8, 17, 16,  9, 12, 25, 31,  0, 11, 21, 22],
      dtype=int64)

In [45]:
#We can see there is a 0, which doesn't make sense. Let's replace it with 15 (half of the month)

data.iday=data.iday.replace({0:15})

In [46]:
data['date']=data.iyear.astype(str)+'-'+data.imonth.astype(str)+'-'+data.iday.astype(str)

In [47]:
data['date']=pd.to_datetime(data['date'])

In [48]:
#Now we are ready to analyze the evolution of terrorism in Egypt over time

Let's now check the columns of numerical values to spot any outlier or weird data

In [49]:
data.select_dtypes(include=np.number).describe()

,Unnamed: 0,eventid,iyear,imonth,iday,extended,country,region,latitude,longitude,...,guncertain1,individual,weaptype1,nkill,property,ishostkid,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
count,2236.000000,2.236000e+03,2236.000000,2236.000000,2236.000000,2236.000000,2236.0,2236.0,2236.000000,2236.000000,...,2236.000000,2236.000000,2236.000000,2236.000000,2236.000000,2236.000000,2236.000000,2236.000000,2236.000000,2236.000000
mean,125515.788014,2.010171e+11,2010.105993,6.325581,15.864043,0.023256,60.0,10.0,29.901316,32.355726,...,0.025939,0.001342,6.073345,1.695886,-0.622540,0.021467,-5.347943,-5.198569,0.034884,-5.032200
std,41623.896427,9.132232e+08,9.134488,3.353116,8.763782,0.150749,0.0,0.0,3.125482,3.328246,...,0.158989,0.036613,1.826228,9.093971,3.049291,0.426719,4.458173,4.647668,0.183526,4.688301
min,133.000000,1.970031e+11,1970.000000,1.000000,1.000000,0.000000,60.0,10.0,0.000000,0.000000,...,0.000000,0.000000,5.000000,0.000000,-9.000000,-9.000000,-9.000000,-9.000000,0.000000,-9.000000
25%,114495.750000,2.013022e+11,2013.000000,3.000000,8.000000,0.000000,60.0,10.0,29.966815,31.185192,...,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,-9.000000,-9.000000,0.000000,-9.000000
50%,140169.500000,2.014111e+11,2014.000000,7.000000,16.000000,0.000000,60.0,10.0,30.968777,33.617577,...,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,-9.000000,-9.000000,0.000000,-9.000000
75%,154526.000000,2.015102e+11,2015.000000,9.000000,23.250000,0.000000,60.0,10.0,31.127890,33.803276,...,0.000000,0.000000,6.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,181625.000000,2.017123e+11,2017.000000,12.000000,31.000000,1.000000,60.0,10.0,31.441411,34.899447,...,1.000000,1.000000,13.000000,311.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
#We also see that the country and region are unique. Indeed, it is all Egypt and MENA region. Let's erase it!
to_drop=['region','region_txt','country','country_txt']

data.drop(columns=to_drop,inplace=True)

In addition, we noticed that some provstate have different names along the dataset. Let's harmonize them

In [52]:
to_replace={
            'Al Iskandariyah (Alexandria)':'Alexandria', 
            'Al Minya (Governorate)':'Minya',
            'Al Qahirah (Cairo)':'Cairo', 
            'Al Qahirah (Governorate)':'Cairo',
            'Alexandria (Governorate)':'Alexandria', 
            'Cairo (Governorate)':'Cairo',
            'North Sinai Governorate':'North Sinai', 
            'Qena':'Quena',
            "Shamal Sina' (Governorate)":'Shamal Sina',
             'South Sinai (Governorate)':'South Sinai',
            'South Sinai Governorate':'South Sinai'
            }
data['provstate']=data['provstate'].replace(to_replace=to_replace)

In [53]:
#Besides that, I can't spot anything strange...let's consider our dataset clean

data.to_csv(r'C:\Users\lenovo\Desktop\Projects DATA\Egypt_terrorism_dataset\clean_data.csv')